### RNN 

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns   
import tensorflow as tf
from tensorflow import keras
from sklearn import (
    linear_model, metrics, neural_network, pipeline, model_selection
)
from sklearn.impute import SimpleImputer

from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.linear_model import Lasso
from sklearn.preprocessing import MinMaxScaler
from keras.models import Sequential
from keras.layers import Dense, Dropout, SimpleRNN, LSTM
from sklearn.metrics import mean_squared_error, accuracy_score, precision_score, recall_score, f1_score
from sklearn.model_selection import TimeSeriesSplit

In [2]:
# import the data
netflix = pd.read_csv('netflix_model_ready.csv')

# Change "date" column to ensure datetime type and drop weekends
netflix["date"] = pd.to_datetime(netflix["date"])

In [6]:
# Time series train/test split
tss= TimeSeriesSplit(n_splits=2)

# Keep all features, but remove the five target variables
X = netflix.iloc[:, :-5]
X = X.set_index("date")

# Create series for target 
y = netflix.iloc[:, -3]

# Split tranining and test data by date
for train_index, test_index in tss.split(X):
    X_train, X_test = X.iloc[train_index, :], X.iloc[test_index,:]
    y_train, y_test = y.iloc[train_index], y.iloc[test_index]

In [7]:
# Standardize the data
scaler = MinMaxScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.fit_transform(X_test)

# PCA feature reduction. 20 components is enough to explain ~85% of the variance (refer to Feture_selection notebook)
sklearn_pca = PCA(n_components=20)
X_train_pca = sklearn_pca.fit_transform(X_train)
X_test_pca = sklearn_pca.transform(X_test)

In [8]:
X_train_3D = np.reshape(X_train_pca, (X_train_pca.shape[0], X_train_pca.shape[1], 1))
X_test_3D = np.reshape(X_test_pca, (X_test_pca.shape[0], X_test_pca.shape[1], 1))


In [13]:
# Construct Long-Short Term Memory (LSTM) model
lstm = Sequential()
lstm.add(LSTM(units = 20, return_sequences = True, input_shape = (X_train_pca.shape[1], 1))) 
lstm.add(LSTM(units = 20, return_sequences = True))
lstm.add(LSTM(units = 20, return_sequences = True))
lstm.add(LSTM(units = 20))
lstm.add(Dense(units = 1))
lstm.compile(optimizer = "adam", loss = "binary_crossentropy", metrics = ["accuracy"])

In [14]:
# fit the model
lstm.fit(X_train_3D, 
        y_train, 
        epochs = 10, 
        validation_data=(X_test_3D, y_test),
        batch_size = 5)

Epoch 1/10
97/97 [==============================] - 8s 35ms/step - loss: 0.9833 - accuracy: 0.4844 - val_loss: 0.7003 - val_accuracy: 0.4833
Epoch 2/10
97/97 [==============================] - 3s 26ms/step - loss: 0.7009 - accuracy: 0.4782 - val_loss: 0.6940 - val_accuracy: 0.5125
Epoch 3/10
97/97 [==============================] - 2s 25ms/step - loss: 0.6987 - accuracy: 0.5114 - val_loss: 0.7007 - val_accuracy: 0.4750
Epoch 4/10
97/97 [==============================] - 3s 26ms/step - loss: 0.7093 - accuracy: 0.4782 - val_loss: 0.7111 - val_accuracy: 0.5208
Epoch 5/10
97/97 [==============================] - 2s 24ms/step - loss: 0.7127 - accuracy: 0.4990 - val_loss: 0.6970 - val_accuracy: 0.4625
Epoch 6/10
97/97 [==============================] - 3s 26ms/step - loss: 0.7073 - accuracy: 0.4740 - val_loss: 0.6931 - val_accuracy: 0.5208
Epoch 7/10
97/97 [==============================] - 2s 26ms/step - loss: 0.6976 - accuracy: 0.5260 - val_loss: 0.7024 - val_accuracy: 0.4708
Epoch 8/10
97